# Voilà Workshop

## Create a new environment

```bash
git clone https://github.com/jtpio/pydata-heidelberg-workshop
cd pydata-heidelberg-workshop/

conda env create
```

This will install a few useful packages for creating dashboards.

[Optional] To install JupyterLab extensions:

```bash
chmod +x ./postBuild
```

Then activate the environment and install other packages you would like to use:

```bash
conda activate voila-starter-pack
conda install -c conda-forge pandas scipy ...
```

## Create a new Jupyter notebook

### Start from scratch

You can create a new notebook from scratch by adding new code cells below:

In [ ]:
print('hello world')

### Use your own notebook

If you have a notebook you would like to use, go ahead!

For example we could take the [Gradient Descent](https://github.com/voila-dashboards/voila-material/blob/master/gradient_descent.ipynb) notebook from the gallery.

### Start from an example

Otherwise, here is a example that uses `pandas`, `ipyleaflet` and `bqplot`: 

In [ ]:
import os
import json

import numpy as np
import pandas as pd

from ipywidgets import Dropdown

from bqplot import Lines, Figure, LinearScale, DateScale, Axis

from ipyleaflet import Map, GeoJSON, WidgetControl, FullScreenControl

In [ ]:
data = pd.read_json(os.path.abspath('nations.json'))

In [ ]:
def clean_data(data):
    for column in ['income', 'lifeExpectancy', 'population']:
        data = data.drop(data[data[column].apply(len) <= 4].index)
    return data

def extrap_interp(data):
    data = np.array(data)
    x_range = np.arange(1800, 2009, 1.)
    y_range = np.interp(x_range, data[:, 0], data[:, 1])
    return y_range

def extrap_data(data):
    for column in ['income', 'lifeExpectancy', 'population']:
        data[column] = data[column].apply(extrap_interp)
    return data

In [ ]:
data = clean_data(data)
data = extrap_data(data)

date_start = pd.datetime(1800, 12, 31)
date_end = pd.datetime(2009, 12, 31)

date_scale = DateScale(min=date_start, max=date_end)

date_data = pd.date_range(start=date_start, end=date_end, freq='A', normalize=True)

country_name = 'Angola'
data_name = 'income'

x_data = data[data.name == country_name][data_name].values[0]

x_scale = LinearScale()

lines = Lines(x=date_data, y=x_data, scales={'x': date_scale, 'y': x_scale})

ax_x = Axis(label='Year', scale=date_scale, num_ticks=10, tick_format='%Y')
ax_y = Axis(label=data_name.capitalize(), scale=x_scale, orientation='vertical', side='left')

figure = Figure(axes=[ax_x, ax_y], title=country_name, marks=[lines], animation_duration=500,
                layout={'height': '300px', 'width': '600px'})

In [ ]:
def update_figure(country_name, data_name):
    lines.y = data[data.name == country_name][data_name].values[0]
    ax_y.label = data_name.capitalize()
    figure.title = country_name

In [ ]:
with open('./countries.geo.json') as f:
    countries = json.load(f)

In [ ]:
m = Map(zoom=3)

geo = GeoJSON(data=countries, style={'fillColor': 'white', 'weight': 0.5}, hover_style={'fillColor': '#1f77b4'}, name='Countries')
m.add_layer(geo)

m.layout.height = '800px'

m

In [ ]:
m.add_control(FullScreenControl())

In [ ]:
widget_control1 = WidgetControl(widget=figure, position='bottomright')

m.add_control(widget_control1)

def on_hover(event, feature, **kwargs):
    global country_name

    country_name = feature['properties']['name']
    update_figure(country_name, data_name)

geo.on_hover(on_hover)

In [ ]:
dropdown = Dropdown(
    options=['income', 'population', 'lifeExpectancy'],
    value=data_name,
    description='Plotting:'
)

def on_click(change):
    global data_name

    data_name = change['new']
    update_figure(country_name, data_name)

dropdown.observe(on_click, 'value')

widget_control2 = WidgetControl(widget=dropdown, position='bottomleft')

m.add_control(widget_control2)

## Test the dashboard in JupyterLab or Classic Notebook

Use the JupyterLab extension or Classic Notebook extension to test the Voilà application.

### JupyterLab Preview Extension

![render-with-voila](./img/render-with-voila.gif)

## Classic Notebook Extension

![nbextension](img/voila-nbextension-button.png)

## [Optional] Serve the dashboard using the `voila` CLI

If you would like to serve the dashboard as a separate application, open a new terminal and run:

```bash
voila workshop.ipynb
```

## Choose a template

Here are a few templates you can choose from:

- https://github.com/voila-dashboards/voila-reveal

- https://github.com/voila-dashboards/voila-material

- https://github.com/voila-dashboards/voila-vuetify

- https://github.com/voila-dashboards/voila-gridstack


For simplicity and elegance, we can go for `voila-material`.

Then open a new terminal and run the following command:

```bash
voila examples/gradient_descent.ipynb --template=material 
```

## Deployment

Finally, we would like to deploy and expose our dashboard to the public!

The documentation provides several options for deploying dashboards: https://voila.readthedocs.io/en/latest/deploy.html

Here we will explore 3 options:

- ngrok
- Binder
- Heroku

### ngrok

To quickly share your notebook with anyone, follow the steps at: https://voila.readthedocs.io/en/latest/deploy.html#sharing-voila-applications-with-ngrok

### Binder

To deploy the dashboard to Binder, follow the steps at: https://voila.readthedocs.io/en/latest/deploy.html#deployment-on-binder

### Heroku

To deploy the dashboard to Heroku, follow the steps at: https://voila.readthedocs.io/en/latest/deploy.html#deployment-on-heroku

## Add the dashboard to the gallery?

If you worked on your own notebook, and deployed it to Binder, maybe you would like to add it to the Gallery?

If so, follow the steps at https://github.com/voila-gallery/voila-gallery.github.io#contributing-new-examples!